<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/20210903load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.4.2

     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 37.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.8 MB/s 


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import urllib.request
import logging
import tensorflow_addons as tfa

In [5]:
logging.basicConfig(level=logging.ERROR)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/0902_load_model_test/lucy_data.csv', encoding='cp949')

#라벨 데이터 category 정수로 변환

data_colname = 'contents'
label_colname = 'category'

encoder = LabelEncoder()
encoder.fit(dataset[label_colname])
dataset[label_colname] = encoder.transform(dataset[label_colname])


#이어서 contents 전처리(정제)
dataset[data_colname] = dataset[data_colname].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("무단전재 및 재배포 금지"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("©"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("ⓒ"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("저작권자"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
dataset[data_colname] = dataset[data_colname].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
dataset[data_colname] = dataset[data_colname].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
dataset[data_colname] = dataset[data_colname].str.replace('\"', "",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
dataset[data_colname] = dataset[data_colname].str.replace("\n"," ")
dataset[data_colname] = dataset[data_colname].str.replace("\r"," ")
dataset[data_colname] = dataset[data_colname].str.replace("\t"," ")
dataset[data_colname] = dataset[data_colname].str.replace( "\’" , "", regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("[ ]{2,}"," ",regex=True)

dataset.head()

,Unnamed: 0,id,contents,category,category_val,predict,predict_tag
0,0,DN000000008778200425,공정소득은 코로나 이후 악화되는 자형 양극화에 대비하자는 것 반박 야권의 대선 주...,8,정치,8,정치
1,1,DN000000008808263362,공동대표 및 정책자문위원 등 임명장 수여 내 삶을 지켜주는 나라 신복지 충남포럼 ...,8,정치,8,정치
2,2,DN000000008791388703,바르셀로나 참가 인텔 기가테라 커뮤니케이션즈 를 오픈랜 생태계를 통하여 구축 및 ...,0,IT/과학,0,IT/과학
3,3,DN000000008853835944,더불어민주당 신영대 의원 이 민주당의 혁신과 흥행을 담당할 대선경선기획단에 인선됐...,8,정치,8,정치
4,4,DN000000008834209192,지난 일 미 멕시코 국경지대인 텍사스주 엘패소와 멕시코 시우대드후아레스 사이에서 ...,4,사회,4,사회


In [11]:
num_epochs = 5
num_batch = 16

In [20]:
warmup_ratio = 0.1

t_total = len(dataset) * num_epochs
warmup_step = int(t_total * warmup_ratio)
initializer_range = 0.2

In [17]:
max_seq_len = 128
LR = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=2e-5, first_decay_steps=warmup_step)

In [18]:
def create_model():
  model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

  input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
  attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
  token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

  outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])
  pooled_output = outputs[1]

  optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
  
  pooled_output = tf.keras.layers.Dropout(0.5)(pooled_output)
  prediction = tf.keras.layers.Dense(9, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=initializer_range))(pooled_output)
  cls_model = tf.keras.Model([input_ids_layer, attention_masks_layer, token_type_ids_layer], prediction)
  cls_model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
  cls_model.summary()

  return cls_model

In [23]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights('/content/drive/MyDrive/0902_load_model_test/best_model')

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_5 (TFBertModel)   TFBaseModelOutputWit 92186880    input_13[0][0]                   
                                                                 input_14[0][0]             

In [24]:
# 토크나이저 코드 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py", filename="tokenization_kobert.py")

('tokenization_kobert.py', <http.client.HTTPMessage at 0x7fdb48c720d0>)

In [25]:
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [26]:
def sentence_prediction(example):
    global tokenizer

    input_ids, attention_masks, token_type_ids = [], [], []

    input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
    # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
    # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
    token_type_id = [0] * max_seq_len
  
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

    input_ids = np.array(input_ids)
    attention_masks = np.array(attention_masks)
    token_type_ids = np.array(token_type_ids)
    return [input_ids, attention_masks, token_type_ids]

def evaluation_predict(sentence):
    data_x = sentence_prediction(sentence)
    predict = model.predict(data_x)

    print('예측 결과 수치', predict)
    print(predict)
    predict_answer = np.argmax(predict[0])
    predict_value = predict[0][predict_answer]
    
    if predict_answer == 0:
      print("(IT/과학 확률 : %.2f) IT/과학 뉴스입니다." % (1-predict_value))
      return 0
    elif predict_answer == 1:
      print("(경제 확률 : %.2f) 경제 뉴스입니다." % predict_value)
      return 1
    elif predict_answer == 2:
      print("(문화 확률 : %.2f) 문화 뉴스입니다." % predict_value)
      return 2
    elif predict_answer == 3:
      print("(미용/건강 확률 : %.2f) 미용/건강 뉴스입니다." % predict_value)
      return 3
    elif predict_answer == 4:
      print("(사회 확률 : %.2f) 사회 뉴스입니다." % predict_value)
      return 4
    elif predict_answer == 5:
      print("(생활 확률 : %.2f) 생활 뉴스입니다." % predict_value)
      return 5
    elif predict_answer == 6:
      print("(스포츠 확률 : %.2f) 스포츠 뉴스입니다." % predict_value)
      return 6
    elif predict_answer == 7:
      print("(연예 확률 : %.2f) 연예 뉴스입니다." % predict_value)
      return 7
    elif predict_answer == 8:
      print("(정치 확률 : %.2f) 정치 뉴스입니다." % predict_value)
      return 8

In [27]:
evaluation_predict('정치 뉴스입니다.')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


예측 결과 수치 [[0.00914906 0.00238315 0.03744877 0.00852321 0.1939369  0.00968546
  0.06027992 0.13331015 0.5452834 ]]
[[0.00914906 0.00238315 0.03744877 0.00852321 0.1939369  0.00968546
  0.06027992 0.13331015 0.5452834 ]]
(정치 확률 : 0.55) 정치 뉴스입니다.


8